In [2]:
import pandas_datareader.data as reader
import pandas as pd
import numpy as np
import datetime
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")

In [3]:
def stepwise_regression(X, y,
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False

        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        
        for new_column in excluded:

            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        
        worst_pval = model.pvalues.max()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(new_pval.index[best_feature])
            changed=True
            if verbose:
                print(f'Add #{new_pval.index[best_feature]}# with p-value {best_pval} ')
            
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        worst_pval = model.pvalues.max()
        if worst_pval > threshold_out:
            worst_feature = model.pvalues.argmax()
            included.remove(model.pvalues.index[worst_feature])
            changed=True
        
            if verbose:
                print(f'Remove #{new_pval.index[worst_feature]}# with p-value {worst_pval} ')
        if not changed:
            break

    return included


In [4]:
fundamentals = pd.read_csv("data/fundamentals.csv")# import factor data
fundamentals = fundamentals.rename(columns={'Ticker Symbol':'symbol',"Period Ending":"date"})
fundamentals.set_index("date",inplace = True) #set the data to index and convert into datatime format
fundamentals.index = pd.to_datetime(fundamentals.index)
fundamentals.reset_index(inplace = True)
prices = pd.read_csv("data/prices-split-adjusted.csv")# import prices data
prices.set_index("date",inplace = True) #set the data to index and convert into datatime format
prices.index = pd.to_datetime(prices.index)
prices['ret'] = prices.groupby("symbol")['close'].pct_change() #use the pct change
prices.dropna(inplace = True)
year_ret = prices.groupby("symbol")["ret"].resample('Y').agg(lambda x : (x+1).prod()-1)
year_ret = pd.DataFrame(year_ret)
year_ret.reset_index(inplace = True)

year_ret

,symbol,date,ret
0,A,2010-12-31,0.323642
1,A,2011-12-31,-0.156891
2,A,2012-12-31,0.172058
3,A,2013-12-31,0.396922
4,A,2014-12-31,0.000771
...,...,...,...
3385,ZION,2016-12-31,0.576557
3386,ZTS,2013-12-31,0.054176
3387,ZTS,2014-12-31,0.316305
3388,ZTS,2015-12-31,0.113642


In [5]:
data_tofit = pd.merge(year_ret,fundamentals)
data_tofit.dropna(inplace = True)
y = data_tofit['ret']
X = data_tofit.iloc[:,4:]
param = stepwise_regression(X,y)


Add #Non-Recurring Items# with p-value 1.2347610960765116e-10 
Add #Cash Ratio# with p-value 0.00012353154433039775 
Add #Liabilities# with p-value 0.0004496578544162408 
Add #Long-Term Debt# with p-value 5.954457632557428e-05 
Add #Sales, General and Admin.# with p-value 0.007004890853336838 
Add #Add'l income/expense items# with p-value 0.004166590529167387 
Add #Net Income Adjustments# with p-value 4.0833933788168954e-05 
Add #Effect of Exchange Rate# with p-value 0.0015723876158670652 
Remove #Sale and Purchase of Stock# with p-value 0.07839323240721335 


In [6]:
model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[param]))).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ret   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.130
Method:                 Least Squares   F-statistic:                     18.98
Date:                Fri, 04 Feb 2022   Prob (F-statistic):           1.29e-23
Time:                        00:02:02   Log-Likelihood:                -154.67
No. Observations:                 845   AIC:                             325.3
Df Residuals:                     837   BIC:                             363.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const                          0.1561      0.014     10.934      0.000       0.128       0.184
Cash Ratio                     0.0004      0.000      3.820      0.000       0.000       0.001
Liabilities                 5.587e-11   9.77e-12      5.722      0.000    3.67e-11     7.5e-11
Long-Term Debt             -4.028e-12   9.66e-13     -4.172      0.000   -5.92e-12   -2.13e-12
Sales, General and Admin.   9.325e-12   1.71e-12      5.448      0.000    5.97e-12    1.27e-11
Add'l income/expense items -6.753e-11   1.24e-11     -5.452      0.000   -9.18e-11   -4.32e-11
Net Income Adjustments     -4.616e-11   6.66e-12     -6.935      0.000   -5.92e-11   -3.31e-11
Effect of Exchange Rate     2.621e-10   7.87e-11      3.331      0.001    1.08e-10    4.17e-10
==============================================================================
Omnibus:                      306.019   Durbin-Watson:                   2.076
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3149.435
Skew:                           1.339   Prob(JB):                         0.00
Kurtosis:                      12.071   Cond. No.                     2.21e+10
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.21e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""